# 순차 데이터(Sequential Data)
- 순서가 의미가 있으며, 순서가 달라질 경우 의미가 손상되는 데이터를 순차 데이터라고 한다.
- 시간적 의미가 있는 경우 Temporal Sequence라고 하며, 일정한 시간차라면 Time Series라고 한다.

### 순차 데이터를 어떻게 학습하고 처리할 수 있을까?

# 기억 시스템(Memory System)
- 올바른 대답을 하려면 입력을 받을 때 마다 그 내용을 '기억'할 수 있어야 한다.
- 이전 입력을 기억하지 않는 시스템은 무기억 시스템이라 한다.

# Vanila RNN
- 얕은 신경망 구조에 '순환'이 추가된 것으로 이해할 수 있다.
- 기억 시스템이므로, RNN의 출력은 이전의 모든 입력에 영향을 받는다.

### 순환 신경망은 기본 역전파 학습법으로 학습할 수 없다. 어떻게 학습해야할까?

# 순차 데이터셋의 구조
- 입력 또는 출력 중 하나라도 순차 데이터라면, 순환 신경망을 이용해 학습할 수 있다.

# 다중 입력에 대해 단일 출력을 하는 경우..


# 순방향 추론
- 입력을 순차적으로 입력하여 마지막 입력시의 출력을 사용

# 시간 펼침 역전파
- 역전파와 동일하게, 시간적으로 펼쳐 둔 상태에서 역전파를 한다.
- 이 때, 시간적으로 펼쳐진 변수들은 동일한 변수라는 점에 유의해야 한다.

# 단일 입력, 다중 출력
- 단일 입력에 대해 다중 출력인 경우, 입력이 한 번 들어온 이후 여러 개의 출력을 낸다.
- 실제로는 입력을 넣고 계산해야 하기 때문에 미리 약속된 입력을 넣어준다.

# 다중 입력, 다중 출력
- 입력과 출력이 매 Time-Step 마다 이루어지는 경우, 
> 동영상의 프레임 별 분류를 예로 들 수 있다.

# 기울기 소실 문제
- 어떤 입력의 정보가 사용되는 시점이 차이가 많이 날 경우, 학습 능력이 저하된다.

# LSTM(Long Short-Term Memory)
- 기억할 것은 오래 기억하고, 잊을 것은 빨리 잊는다.

    ### 1) Cell State 
        - 기억을 오랫동안 유지할 수 있는 구조 
        > 새로운 특징을 덧셈으로 받는 구조 

    ### 2) Hidden State 
        - 계층의 출력 / 다음 타임 스텝으로 넘어가는 정보
    
    ### 3) Forget Gate
        - Sigmoid 활성 함수로, 0~1의 출력 값을 가짐.
        - Cell State에 이를 곱해 주어서 '얼만큼 잊을지'를 결정.

    ### 4) Input Gate
        - Sigmoid 활성 함수로, 0~1의 출력 값을 가짐
        - 새롭게 추출한 특징을 얼만큼 사용할 지 결정.
         
    ### 5) Output Gate
        - Sigmoid 활성 함수로, 0~1의 출력 값을 가짐
        - Cell로부터 출력을 얼마나 내보낼지 결정하는 역할.
        
> RNN과 달리, Cell State가 있어서 '기억'에 관한 부분을 전담한다.<br>
> 하지만, 학습 매개변수가 많고 학습 노하우가 중요한 편이다.

# GRU (Gated Recurrent Unit)
- LSTM을 간소화한 버전이라고 할 수 있는 GRU.

### GRU의 구조
- Cell State가 없고, Hidden State만 존재한다.
- Forget Gate와 Input Gate를 결합하였다.
- Reset Gate를 추가하였다.

### GRU의 Forget Gate & Input Gate
- LSTM과 동일한 Forget Gate를 사용한다.
- Forget Gate를 1에서 빼서 Input Gate로 사용하였다.

### Reset Gate
- Sigmoid 활성 함수로, 0~1의 값을 가진다.
- 이전 Hidden state를 얼마나 사용할지 정하는 역할
- 0에 가까운 값이 되면 'Reset'이 된다. (ex. 새 문장의 시작)

# 순환신경망 구현 및 학습

In [2]:
import tensorflow as tf
import numpy as np

# 하이퍼 파라미터

In [9]:
EPOCHS = 10
NUM_WORDS = 10000

# 모델 정의

In [10]:
np.__version__

'1.19.5'

In [11]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.lstm = tf.keras.layers.LSTM(32)
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')
# binary classification을 사용하기에 sigmoid 함수 사용

    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.lstm(x)
        return self.dense(x)

# IMDB 데이터셋 준비

In [12]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)
# x는 영화의 평, y는 긍정/부정을 나타내는 label.

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, value=0, padding='pre', maxlen=32)
# 32길이로 잘라주고 패딩이 필요하면 앞 쪽에서 0으로 패딩
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, value=0, padding='pre', maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 모델 생성

In [13]:
model = MyModel()

model.compile(optimizer='adam',
loss='binary_crossentropy', # Multiclass가 아니기에 binary
metrics=['accuracy'])

# 모델 학습

In [14]:
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 0.5713 - accuracy: 0.6776 - val_loss: 0.4397 - val_accuracy: 0.7936
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.3802 - accuracy: 0.8276 - val_loss: 0.4673 - val_accuracy: 0.7799
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.3294 - accuracy: 0.8584 - val_loss: 0.4852 - val_accuracy: 0.7775
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2904 - accuracy: 0.8804 - val_loss: 0.5505 - val_accuracy: 0.7667
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2419 - accuracy: 0.9034 - val_loss: 0.5629 - val_accuracy: 0.7722
Epoch 6/10
782/782 [==============================] - 3s 3ms/step - loss: 0.1957 - accuracy: 0.9240 - val_loss: 0.6413 - val_accuracy: 0.7662
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1568 - accuracy: 0.9413 - val_loss: 0.6937 - val_accuracy: 0.7609
Epoch 

- loss는 줄어드는 반면, val_loss는 점점 올라가 overfitting 현상 발생.
- 이런 상황일 때 GRU를 사용해보자!

# VanilaRNN 사용해보기

In [19]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.rnn = tf.keras.layers.SimpleRNN(32)
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')
# binary classification을 사용하기에 sigmoid 함수 사용

    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

In [20]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)
# x는 영화의 평, y는 긍정/부정을 나타내는 label.

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, value=0, padding='pre', maxlen=32)
# 32길이로 잘라주고 패딩이 필요하면 앞 쪽에서 0으로 패딩
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, value=0, padding='pre', maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [21]:
model = MyModel()

model.compile(optimizer='adam',
loss='binary_crossentropy', # Multiclass가 아니기에 binary
metrics=['accuracy'])

In [22]:
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
782/782 [==============================] - 6s 7ms/step - loss: 0.6294 - accuracy: 0.6140 - val_loss: 0.4608 - val_accuracy: 0.7867
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3863 - accuracy: 0.8270 - val_loss: 0.4784 - val_accuracy: 0.7727
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2235 - accuracy: 0.9128 - val_loss: 0.6002 - val_accuracy: 0.7565
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0988 - accuracy: 0.9670 - val_loss: 0.8022 - val_accuracy: 0.7528
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0398 - accuracy: 0.9881 - val_loss: 1.0394 - val_accuracy: 0.7306
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0256 - accuracy: 0.9923 - val_loss: 1.1693 - val_accuracy: 0.7435
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0177 - accuracy: 0.9955 - val_loss: 1.2378 - val_accuracy: 0.7403
Epoch 